In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\snow dispute.mp4',
 'video_footages\\Surveillance camera captures tanks entering Ukraine via Belarus border.mp4',
 'video_footages\\trail cam video creatures of the night.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4',
 'video_footages\\차흐르듯 (1).mp4',
 'video_footages\\차흐르듯.mp4']

In [7]:
video=video_files[1]
cap = cv2.VideoCapture(video)
ret, frame = cap.read()
#h, w, l = frame.shape
n_h = 360#int(h/2)
n_w = 600#int(w/2)

#getting background model from video:
frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
# we will store the frames in array
frames = []
for idx in frame_indices:
    # set the frame id to read that particular frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ret, frame = cap.read()
    frames.append(frame)
# calculate the median
median_frame = np.median(frames, axis=0).astype(np.uint8)
background = cv2.resize(median_frame, (n_w, n_h))
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)


frame_count = 0
consecutive_frame = 8

cap = cv2.VideoCapture(video)

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        frame = cv2.resize(frame, (n_w, n_h))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if frame_count % consecutive_frame == 0 or frame_count == 1:
            frame_diff_list = []
        
        #delta_frame = get_mask(old_gray, gray)
        frame_diff = cv2.absdiff(gray, background)
        ret, thresh = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
        dilate_frame = cv2.dilate(thresh, None, iterations=2)
        frame_diff_list.append(dilate_frame)

        if len(frame_diff_list) == consecutive_frame:
            # add all the frames in the `frame_diff_list`
            sum_frames = sum(frame_diff_list)

            # find the contours around the white segmented areas
            contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # draw the contours, not strictly necessary
            #for i, cnt in enumerate(contours):
            #    cv2.drawContours(frame, contours, i, (0, 0, 255), 3)

            for contour in contours:
                # continue through the loop if contour area is less than 500...
                # ... helps in removing noise detection
                if cv2.contourArea(contour) < 500:
                    continue

                # get the xmin, ymin, width, and height coordinates from the contours
                (x, y, w, h) = cv2.boundingRect(contour)
                # draw the bounding boxes
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            #cv2.imshow('mask', dilate_frame)
            cv2.imshow('Detected Objects', frame)
            cv2.imshow('background', background)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break

    else:
        break

        
 
cap.release()
 
cv2.destroyAllWindows()

# Loop through the whole videos file

In [8]:
for video in video_files:
    cap = cv2.VideoCapture(video)
    ret, frame = cap.read()
    #h, w, l = frame.shape
    n_h = 360#int(h/2)
    n_w = 600#int(w/2)
    
    #getting background model from video:
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    background = cv2.resize(median_frame, (n_w, n_h))
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    
    frame_count = 0
    consecutive_frame = 8
    
    cap = cv2.VideoCapture(video)
    
    while(cap.isOpened()):
        
        ret, frame = cap.read()
        if ret == True:
            frame_count += 1
            frame = cv2.resize(frame, (n_w, n_h))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
            if frame_count % consecutive_frame == 0 or frame_count == 1:
                frame_diff_list = []
            
            #delta_frame = get_mask(old_gray, gray)
            frame_diff = cv2.absdiff(gray, background)
            ret, thresh = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
            dilate_frame = cv2.dilate(thresh, None, iterations=2)
            frame_diff_list.append(dilate_frame)
    
            if len(frame_diff_list) == consecutive_frame:
                # add all the frames in the `frame_diff_list`
                sum_frames = sum(frame_diff_list)
    
                # find the contours around the white segmented areas
                contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                # draw the contours, not strictly necessary
                #for i, cnt in enumerate(contours):
                #    cv2.drawContours(frame, contours, i, (0, 0, 255), 3)
    
                for contour in contours:
                    # continue through the loop if contour area is less than 500...
                    # ... helps in removing noise detection
                    if cv2.contourArea(contour) < 500:
                        continue
    
                    # get the xmin, ymin, width, and height coordinates from the contours
                    (x, y, w, h) = cv2.boundingRect(contour)
                    # draw the bounding boxes
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
                #cv2.imshow('mask', dilate_frame)
                cv2.imshow('Detected Objects', frame)
                cv2.imshow('background', background)
                if cv2.waitKey(60) & 0xFF == ord('q'):
                    break
    
        else:
            break
    
            
     
    cap.release()
     
    cv2.destroyAllWindows()